In [65]:
using OhMyJulia
using DataFrames

## Setup

In [126]:
function parse_path(file)
    pathdict = Dict{Pair{String, String}, Vector{Pair{Vector{Pair{String, String}}, Float64}}}()
    
    cp = nothing
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        if m != nothing
            pathdict[car(m) => cadr(m)] = []
            cp = pathdict[car(m) => cadr(m)]
            continue
        end
        
        s = split(line, '@')
        if length(s) == 2
            p = parse(car(s))
            p = map(x->Pair(string.(x.args)...), p.args)
            w = parse(cadr(s))
            push!(cp, p => w)
        end
    end
    
    pathdict
end

function parse_topo(file)
    nodes = Set{String}()
    edges = Set{Pair{String, String}}()
    for line in eachline(file)
        m = match(r"([hs]\d+)\s*->\s*([hs]\d+)", line)
        m == nothing && continue
        
        src, dst = car(m), cadr(m)
        push!(nodes, src, dst)
        push!(edges, src => dst)
    end
    collect(nodes), collect(edges)
end

function mkdf(names...)
    DataFrame([[] for i in names], [Symbol(i) for i in names])
end

function cut_path!()
    ref = Dict(k=>length(v) for (k, v) in paths["custom_0"])
    
    for (algo, path) in paths
        for (pair, scheme) in path
            scheme = sort(scheme, rev=true, by=cadr)
            length(scheme) <= ref[pair] && continue
            
            newscheme = scheme[1:ref[pair]]
            factor = sum(cadr.(newscheme))
            path[pair] = [p => w / factor for (p, w) in newscheme]
        end
    end
end

data = "abilene"
nodes, edges = parse_topo("data/topologies/$data.dot")
# algos = [basename(x) for x in all_files("data/results/$data/paths") if match(r"_\d+$", x) != nothing]
algos = [basename(x) for x in all_files("data/results/$data/paths") if endswith(x, "_0")]
paths = Dict(algo => parse_path("data/results/$data/paths/$algo") for algo in algos)
cut_path!()

## avg path length

In [127]:
df = mkdf("algorithm", "average length", "max length")
for (algo, path) in paths
    lens = [length(car(p)) for (k, v) in path for p in v]
    push!(df, [algo[1:end-2] mean(lens) maximum(lens)])
end
df

,algorithm,average length,max length
1,custom,5.25,9
2,semimcfvlb,5.45565,11
3,raeke,5.20161,9
4,semimcfecmp,4.64103,7
5,semimcfraeke,5.64113,12
6,ksp,5.04032,8
7,vlb,5.08468,8
8,semimcfksp,5.69355,11
9,semimcfcustom,5.25,9
10,semimcfedksp,5.25,9


In [ ]:
## one edge failure

In [128]:
df = mkdf("algorithm", "affected path", "affected pair", "affected weight")
for (algo, path) in paths
    apath, apair, aweight = 0, 0, 0
    
    for edge in edges @when startswith(car(edge), "s") && startswith(cadr(edge), "s")
        for (pair, scheme) in path
            pair_affected = false
            
            for (p, w) in scheme @when edge in p
                pair_affected = true
                apath += 1
                aweight += w
            end
            
            apair += pair_affected
        end
    end
    
    push!(df, [algo[1:end-2] apath apair aweight])
end
df

,algorithm,affected path,affected pair,affected weight
1,custom,806,806,427.333
2,semimcfvlb,857,784,494.705
3,raeke,794,776,381.201
4,semimcfecmp,412,388,330.0
5,semimcfraeke,903,825,550.649
6,ksp,754,677,401.333
7,vlb,765,705,367.184
8,semimcfksp,916,840,562.628
9,semimcfcustom,806,806,393.636
10,semimcfedksp,806,806,393.636


## one node failure

df = mkdf("algorithm", "affected path", "affected pair", "affected weight")
for (algo, path) in paths
    apath, apair, aweight = 0, 0, 0
    
    for node in nodes @when startswith(node, "s")
        for (pair, scheme) in path
            pair_affected = false
            
            for (p, w) in scheme @when any(x->node in x, p)
                pair_affected = true
                apath += 1
                aweight += w
            end
            
            apair += pair_affected
        end
    end
    
    push!(df, [algo[1:end-2] apath apair aweight])
end
df